In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [16]:
import numpy as np
from math import sqrt
import matplotlib.pyplot as plt
from sklearn.cluster import SpectralClustering 



#calculate sum of square error
def sse(matrix, clusters, centroids, k):
  sum = [0]*k
  for i in range(k):
    for j in range(len(clusters)):
      if(clusters[j] == i):
        for p in range(len(matrix[j])):
          sum[i] += (matrix[j][p] - centroids[i][p])**2
  return sum

        

#calculate euclidean distance
def euclidean_distance(centroid, row):
  d = 0
  j = 0
  for i in row:
    term = (i - centroid[j])**2
    d += term
    j += 1
  return sqrt(d)



#calculate means for every cluster and update centroids
def mean(matrix, cluster, cl):
  newCentroids = [] 
  numClusters=0  
  
  while(numClusters < cl):
    sum = [0] * len(matrix[0])
    count = 0
    for k in range(len(cluster)):
      if(cluster[k] == numClusters):
        count+=1
        for i in range((len(matrix[k]))):
          sum[i] += matrix[k][i]

    for j in range(len(sum)):
      sum[j] = sum[j]/count

    newCentroids.append(sum)
    numClusters += 1

  return newCentroids


#k-means
def kmeans(matrix, k): 
  centroids = []
  cluster = [0] * len(matrix)
  
  #ramdomly select 5 centroids
  for i in range(k):
    centroids = matrix[np.random.choice(matrix.shape[0], size=k, replace=False), :]


  iteration = 1000
  while(iteration>=0):
    #assignment stage
    for i in range(len(matrix)):
      minDistance = 1000
      for j in range(len(centroids)):
        distance = euclidean_distance(centroids[j], matrix[i]) #measure the distance to every centroid
        if(distance < minDistance):
          minDistance = distance
          cluster[i] = j #assign all data to closest centroid

    #update stage
    centroids = mean(matrix, cluster, k) # Calculate mean for every cluster and update the centroid
    iteration-=1

  return cluster, centroids



def purity(original_data, matrix, clusters, k):
  purity = 0
  max_occurences = 0
  
  for i in range(k): #loop through every cluster
    numPoints = 0 
    answer_occurences = [0]*k
    for index in range(len(clusters)):
      if(clusters[index] == i):
        numPoints += 1 #count number of points in each cluster
        ground_truth = original_data[index][1] #find the actual clustering of the specific row i in ground truth table

        if ground_truth == 1:
          answer_occurences[0] += 1
        elif ground_truth == 2:
          answer_occurences[1] += 1
        elif ground_truth == 3:
          answer_occurences[2] += 1
        elif ground_truth == 4:
          answer_occurences[3] += 1
        elif ground_truth == 5:
          answer_occurences[4] += 1

        elif ground_truth == 6:
          answer_occurences[5] += 1
        elif ground_truth == 7:
          answer_occurences[6] += 1
        elif ground_truth == 8:
          answer_occurences[7] += 1
        elif ground_truth == 9:
          answer_occurences[8] += 1
        elif ground_truth == 10:
          answer_occurences[9] += 1

    max_occurences = max(answer_occurences) 
    purity += (max_occurences/numPoints)

  purity = purity / k
  return purity


def spectral(matrix, k):
  # build the clustering model 
  spectral_model_nn = SpectralClustering(n_clusters = k, affinity ='nearest_neighbors') 
    
  # train the model and store the predicted cluster labels 
  labels_nn = spectral_model_nn.fit_predict(matrix) 
  
  return labels_nn


#MAIN METHOD
if __name__ == "__main__":
  #load files from google drive
  cho_matrix = np.loadtxt('/content/drive/My Drive/cse347_project_data/cho.txt')
  iyer_matrix = np.loadtxt('/content/drive/My Drive/cse347_project_data/iyer.txt')

  #remove outliers in cho
  cleaned_cho = []
  for row in cho_matrix:
    if(row[1] != -1):
      cleaned_cho.append(row)

  #remove outliers in iyer
  cleaned_iyer = []
  for row in iyer_matrix:
    if(row[1] != -1):
      cleaned_iyer.append(row)
  original_iyer = cleaned_iyer

  #remove the first two columns in cho
  cleaned_cho = np.delete(cleaned_cho, 0, 1)
  cleaned_cho = np.delete(cleaned_cho, 0, 1)

  #remove the first two columns in iyer
  cleaned_iyer = np.delete(cleaned_iyer, 0, 1)
  cleaned_iyer = np.delete(cleaned_iyer, 0, 1)

  #normalization for cho
  i = 0
  for row in cleaned_cho:
    minVal = np.min(row)
    maxVal = np.max(row)
    j = 0
    for val in row:
      cleaned_cho[i][j] = (val-minVal) / (maxVal-minVal)
      j+=1
    i+=1

  #normalization for cho
  i = 0
  for row in cleaned_iyer:
    minVal = np.min(row)
    maxVal = np.max(row)
    j = 0
    for val in row:
      cleaned_iyer[i][j] = (val-minVal) / (maxVal-minVal)
      j+=1
    i+=1


  # PROMPT THE USER FOR THE ALGORITHM TO RUN
  choice = input("Please choose the algorithm to run, type '1' for kemeans, '2' for spectral clustering:\n")
  while(choice != '1' and choice != '2'):
    choice = input("Please input a valid number:\n")
  print(f'You entered {choice}')


  ## KMEANS IMPLEMENTATION AND EVALUATION
  if(choice == '1'):
    print("Hint: it could take a minute for the results to show up. Please be patient. ")

    cluster1, centroids1 = kmeans(cleaned_cho, 5)

    sse_cho = sse(cleaned_cho, cluster1, centroids1, 5)
    print("The sume of square errors for each cluster in cho are ")
    for e in range(len(sse_cho)):
      print("cluster[", e+1, "]: ", sse_cho[e])

    purity_cho = purity(cho_matrix, cleaned_cho, cluster1, 5)
    print("The purity for cho with kmeans is ", purity_cho)

    cluster2, centroids2 = kmeans(cleaned_iyer, 10)

    sse_iyer = sse(cleaned_iyer, cluster2, centroids2, 10)
    print("The sume of square errors for each cluster in iyer are ")
    for e in range(len(sse_iyer)):
      print("cluster[", e+1, "]: ", sse_iyer[e])

    purity_iyer = purity(original_iyer, cleaned_iyer, cluster2, 10)
    print("The purity for iyer with kemeans is ", purity_iyer)


  ## SPECTRAL CLUSTERING IMPLEMENTATION AND EVALUATION
  elif(choice == '2'):
    labels_cho = spectral(cleaned_cho, 5)

    centriods_cho = mean(cleaned_cho, labels_cho, 5)
    sse_cho_spec = sse(cleaned_cho, labels_cho, centriods_cho, 5)
    print("The sume of square errors for each cluster in cho are ")
    for e in range(len(sse_cho_spec)):
      print("cluster[", e+1, "]: ", sse_cho_spec[e])

    purity_cho_spec = purity(cho_matrix, cleaned_cho, labels_cho, 5)
    print("The purity for cho with spectral clustering is ", purity_cho_spec)

    labels_iyer = spectral(cleaned_iyer, 10)

    centroids_iyer = mean(cleaned_iyer, labels_iyer, 10)
    sse_iyer_spec = sse(cleaned_iyer, labels_iyer, centroids_iyer, 10)
    print("The sume of square errors for each cluster in iyer are ")
    for e in range(len(sse_iyer_spec)):
      print("cluster[", e+1, "]: ", sse_iyer_spec[e])

    purity_iyer_spec = purity(original_iyer, cleaned_iyer, labels_iyer, 10)
    print("The purity for iyer with spectral clustering is ", purity_iyer_spec)

    


Please choose the algorithm to run, type '1' for kemeans, '2' for spectral clustering:
2
You entered 2
The sume of square errors for each cluster in cho are 
cluster[ 1 ]:  71.52276821989045
cluster[ 2 ]:  46.104077894104805
cluster[ 3 ]:  56.35864362334264
cluster[ 4 ]:  14.325067318684711
cluster[ 5 ]:  41.630182653257265
The purity for cho with spectral clustering is  0.6201342292376775
The sume of square errors for each cluster in iyer are 
cluster[ 1 ]:  7.860131802951841
cluster[ 2 ]:  39.60205227467691
cluster[ 3 ]:  27.441152307317427
cluster[ 4 ]:  14.158167990060816
cluster[ 5 ]:  15.401683516709614
cluster[ 6 ]:  9.08225728335192
cluster[ 7 ]:  7.049835183870017
cluster[ 8 ]:  27.836066642486518
cluster[ 9 ]:  1.2498067718492183
cluster[ 10 ]:  13.956227611115663
The purity for iyer with spectral clustering is  0.7479646935665318
